In [1]:
import numpy as np
import pandas as pd
import joblib
from sklearn import preprocessing

In [2]:
test = pd.read_csv("./test.csv")
sample_submission = pd.read_csv("./sample_submission.csv")

In [3]:
not_important_list = [
    'id',
    'product_code'
]

for s in not_important_list:
  test = test.drop(s, axis=1)

In [4]:
# transform object to int64
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html

test['attribute_1'] = test['attribute_1'].astype(str).apply(lambda a:str(a[-1:]))

le = preprocessing.LabelEncoder()
le.fit(['8', '7', '6', '5'])
test['attribute_1'] = le.transform(list(test['attribute_1'].values))

for i in test.columns[test.dtypes == 'object']:
  le = preprocessing.LabelEncoder()
  le.fit(test[i])
  test[i] = le.transform(list(test[i].values))

In [5]:
test = test.fillna(test.mean())
print(test.isna().sum())

loading           0
attribute_0       0
attribute_1       0
attribute_2       0
attribute_3       0
measurement_0     0
measurement_1     0
measurement_2     0
measurement_3     0
measurement_4     0
measurement_5     0
measurement_6     0
measurement_7     0
measurement_8     0
measurement_9     0
measurement_10    0
measurement_11    0
measurement_12    0
measurement_13    0
measurement_14    0
measurement_15    0
measurement_16    0
measurement_17    0
dtype: int64


In [6]:
models = joblib.load('weight.joblib')

test_preds = np.zeros(len(test))
for model in models:
  test_preds += model.predict_proba(test)[:, 1]

In [7]:
sub = pd.DataFrame({'id': sample_submission.id, 'failure': test_preds / len(models)})
sub.to_csv("submission.csv", index=False)